## Import Libraries

In [1]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split,KFold

## Dataset

In [3]:
#load the dataset
diabetes_dataset = pd.read_csv('/Users/tinonturjamajumder/Downloads/pima-indians-diabetes.csv')
diabetes_dataset.head()

,6,148,72,35,0,33.6,0.627,50,1
0,1,85,66,29,0,26.6,0.351,31,0
1,8,183,64,0,0,23.3,0.672,32,1
2,1,89,66,23,94,28.1,0.167,21,0
3,0,137,40,35,168,43.1,2.288,33,1
4,5,116,74,0,0,25.6,0.201,30,0


In [4]:
#turn the dataset into numpy values
diabet_data = diabetes_dataset.to_numpy()


In [5]:
X = diabet_data[:,:8]
y = diabet_data[:,8]

In [6]:
X.shape,y.shape

((767, 8), (767,))

## Device Agnostic Code

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

## Splitting data

In [8]:
def splitting_data(X,y,test_size):
  X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=test_size)
  return X_train,X_test,y_train,y_test

## Scaling data

In [9]:
def scaled_data(X_train,X_test):
  scaler = MinMaxScaler()
  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)
  return X_train,X_test

## Model

In [10]:
class ClassificationModelv1(nn.Module):
  def __init__(self,l1,l2):
    super().__init__()
    self.layer1 = nn.Linear(in_features=8,
                            out_features=l1)
    self.layer2 = nn.Linear(in_features=l1,
                            out_features=l2)
    self.layer3 = nn.Linear(in_features=l2,
                            out_features = 1)

    #activation function
    self.relu = nn.ReLU()


  def forward(self,x):
    return self.layer3(self.relu(self.layer2(self.relu(self.layer1(x)))))

In [11]:
model_1 = ClassificationModelv1(10,8).to(device)

## Training and Testing

In [12]:
def train_test(X_train,y_train,X_test,y_test,epochs,model,lr = 0.001):
  #scaled the data
  X_train,X_test = scaled_data(X_train,X_test)

  #convert torch to tensors
  X_train = torch.tensor(X_train,dtype = torch.float32).to(device)
  y_train = torch.tensor(y_train,dtype = torch.float32).to(device).squeeze()
  X_test = torch.tensor(X_test,dtype = torch.float32).to(device)
  y_test = torch.tensor(y_test,dtype = torch.float32).to(device).squeeze()
  model = model.to(device)
  train_loss_list = []
  test_loss_list = []
  train_accuracies = []
  test_accuracies =[]
  #loss function
  loss_fn = nn.BCEWithLogitsLoss()

  #optimizer
  optimizer = torch.optim.Adam(model.parameters(),
                               lr = lr)

  #train the model
  for epoch in range(epochs):
    model.train()

    #training data
    train_pred = model(X_train).squeeze()
    train_loss = loss_fn(train_pred,y_train)
    train_loss_list.append(train_loss.item())
    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()
    train_accuracy = ((train_pred.round() == y_train).float().mean()).item()
    train_accuracies.append(train_accuracy)
    #testing the model
    with torch.inference_mode():
      model.eval()
      test_pred = model(X_test).squeeze()
      test_loss = loss_fn(test_pred,y_test)
      test_loss_list.append(test_loss.item())

      test_accuracy = ((test_pred.round() == y_test).float().mean()).item()
      test_accuracies.append(test_accuracy)
    if epoch %10 ==0:
      print(f"Train_loss: {train_loss}  | Test_loss: {test_loss}  | Train_accuracy: {train_accuracy}  | Test_accuracy: {test_accuracy}")

  return train_loss_list,test_loss_list,train_accuracies,test_accuracies


## Task-1 & Task-5

## Task-2